<a href="https://colab.research.google.com/github/Boo90/ai-chatbot/blob/main/%D9%88%D8%A7%D8%AC%D9%87%D8%A9_Titapa_AI_%D8%A7%D9%84%D8%B1%D8%B3%D9%88%D9%85%D9%8A%D8%A9_%D8%A7%D9%84%D9%85%D8%AA%D9%82%D8%AF%D9%85%D8%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import tkinter as tk
from tkinter import messagebox
from datetime import datetime
import json
import threading
import time

# هذا الكود هو واجهة مستخدم رسومية (GUI) متقدمة لنظام Titapa AI.
# تم دمج كل منطق النظام السابق مع واجهة رسومية باستخدام مكتبة Tkinter.

class TitapaGUI(tk.Tk):
    """
    الفئة الرئيسية لواجهة المستخدم الرسومية.
    تدير جميع عناصر الواجهة مثل النوافذ، الأزرار، والملصقات.
    """
    def __init__(self, titapa_core):
        super().__init__()
        self.titapa_core = titapa_core
        self.title("نظام Titapa AI")
        self.geometry("800x600")
        self.configure(bg="#2c3e50")  # لون خلفية داكن وأنيق

        # تهيئة الأيقونات
        self.icon_weather_sunny = "☀️"
        self.icon_weather_cloudy = "☁️"
        self.icon_device_on = "✅"
        self.icon_device_off = "❌"

        self.create_widgets()
        self.update_gui_thread()
        self.titapa_core.gui = self  # ربط الوحدة الأساسية بالواجهة الرسومية

    def create_widgets(self):
        """إنشاء وتنسيق عناصر الواجهة."""
        # إطار علوي للمعلومات الرئيسية
        top_frame = tk.Frame(self, bg="#34495e", padx=20, pady=10)
        top_frame.pack(fill=tk.X)

        self.time_label = tk.Label(top_frame, text=datetime.now().strftime("%H:%M:%S"), font=("Arial", 14), bg="#34495e", fg="#ecf0f1")
        self.time_label.pack(side=tk.LEFT)

        self.user_label = tk.Label(top_frame, text="المستخدم: ضيف", font=("Arial", 14), bg="#34495e", fg="#ecf0f1")
        self.user_label.pack(side=tk.RIGHT)

        # إطار لعرض حالة النظام
        status_frame = tk.Frame(self, bg="#2c3e50", pady=20)
        status_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

        self.status_label = tk.Label(status_frame, text="Titapa AI جاهز للعمل.", font=("Arial", 18, "bold"), bg="#2c3e50", fg="#ecf0f1")
        self.status_label.pack(pady=10)

        # عرض حالة الأجهزة
        self.device_status_label = tk.Label(status_frame, text="", font=("Arial", 12), bg="#2c3e50", fg="#bdc3c7")
        self.device_status_label.pack()

        # عرض حالة الطقس
        self.weather_status_label = tk.Label(status_frame, text="", font=("Arial", 12), bg="#2c3e50", fg="#bdc3c7")
        self.weather_status_label.pack()

        # صندوق عرض المخرجات (شاشة النظام)
        self.output_text = tk.Text(self, wrap=tk.WORD, state=tk.DISABLED, bg="#1c2833", fg="#ecf0f1", font=("Arial", 12), bd=0, highlightthickness=0)
        self.output_text.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        self.output_text.tag_config('system', foreground='#1abc9c')
        self.output_text.tag_config('user', foreground='#3498db')

        # إطار لمدخل الأوامر
        input_frame = tk.Frame(self, bg="#2c3e50", pady=10)
        input_frame.pack(fill=tk.X, side=tk.BOTTOM)

        self.command_entry = tk.Entry(input_frame, font=("Arial", 14), bg="#34495e", fg="#ecf0f1", bd=0, insertbackground="#ecf0f1")
        self.command_entry.bind("<Return>", self.send_command)
        self.command_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=10, ipady=5)
        self.command_entry.focus_set()

        self.send_button = tk.Button(input_frame, text="إرسال", command=self.send_command, font=("Arial", 12, "bold"), bg="#1abc9c", fg="#ffffff", activebackground="#16a085", relief=tk.FLAT)
        self.send_button.pack(side=tk.RIGHT, padx=10, ipadx=10, ipady=5)

    def send_command(self, event=None):
        """
        يتم استدعاؤها عند إرسال أمر من قبل المستخدم.
        """
        command = self.command_entry.get().strip()
        if not command:
            return

        self.display_message(f"أنت: {command}", 'user')
        self.command_entry.delete(0, tk.END)

        # تشغيل الأمر في خيط منفصل لتجنب تجميد الواجهة
        threading.Thread(target=self.titapa_core.process_command, args=(command,)).start()

    def display_message(self, message, tag='system'):
        """
        يعرض الرسائل في شاشة المخرجات.
        """
        self.output_text.config(state=tk.NORMAL)
        self.output_text.insert(tk.END, f"{message}\n", tag)
        self.output_text.see(tk.END)
        self.output_text.config(state=tk.DISABLED)

    def update_gui_thread(self):
        """
        يحدث عناصر الواجهة بشكل دوري.
        """
        self.time_label.config(text=datetime.now().strftime("%H:%M:%S"))

        if self.titapa_core.current_user_id:
            user_name = self.titapa_core.user_profiles.get(self.titapa_core.current_user_id, {}).get("name", "ضيف")
            self.user_label.config(text=f"المستخدم: {user_name}")

        device_status_text = ' | '.join([f"{d}: {self.icon_device_on if s else self.icon_device_off}" for d, s in self.titapa_core.smart_home.devices.items()])
        self.device_status_label.config(text=f"حالة الأجهزة: {device_status_text}")

        city = self.titapa_core.user_profiles.get(self.titapa_core.current_user_id, {}).get("preferences", {}).get("city", None) if self.titapa_core.current_user_id else None
        weather_status_text = self.titapa_core.external_services.get_status(city)
        self.weather_status_label.config(text=f"حالة الطقس: {weather_status_text}")

        # تحديث الواجهة كل ثانية
        self.after(1000, self.update_gui_thread)

# ----------------------------------------
# |        منطق نظام Titapa AI           |
# ----------------------------------------
class TitapaAI:
    def __init__(self):
        self.user_profiles = {}
        self.load_profiles()
        self.current_user_id = None
        self.is_running = True
        self.nlp = NLPController()
        self.gui = None  # سيتم ربط الواجهة الرسومية هنا
        self.smart_home = SmartHomeController()
        self.external_services = ExternalServicesController()
        self.tts = TTSController(self)
        self.modes = {
            "ترفيه": self.entertainment_mode,
            "إنتاجية": self.productivity_mode,
            "منزل ذكي": self.smart_home_mode,
            "صحة": self.health_mode
        }
        self.current_mode = "dashboard"

    def load_profiles(self):
        try:
            with open('user_profiles.json', 'r', encoding='utf-8') as f:
                self.user_profiles = json.load(f)
        except FileNotFoundError:
            self.user_profiles = {
                "user_101": {"name": "أحمد", "preferences": {"mode": "ترفيه", "city": "الرياض"}},
                "user_102": {"name": "سارة", "preferences": {"mode": "إنتاجية", "city": "القاهرة"}}
            }
            self.save_profiles()

    def save_profiles(self):
        with open('user_profiles.json', 'w', encoding='utf-8') as f:
            json.dump(self.user_profiles, f, ensure_ascii=False, indent=4)

    def update_user_preference(self, key, value):
        if self.current_user_id:
            if "preferences" not in self.user_profiles[self.current_user_id]:
                self.user_profiles[self.current_user_id]["preferences"] = {}
            self.user_profiles[self.current_user_id]["preferences"][key] = value
            self.save_profiles()
            self.tts.speak(f"تم تحديث تفضيلك: {key} إلى {value}.")
        else:
            self.tts.speak("عذراً، لا يمكنني تحديث التفضيلات الآن.")

    def run(self):
        # محاكاة التعرف على الوجه عند بدء التشغيل
        self.tts.speak("أدخل معرف المستخدم في الطرفية (user_101 أو user_102).")
        user_id = input("[📸] Titapa: أدخل اسم المستخدم: ").strip()
        if user_id in self.user_profiles:
            self.current_user_id = user_id
            self.tts.speak(f"أهلاً بك يا {self.user_profiles[user_id]['name']}.")
        else:
            self.tts.speak("لم يتم التعرف على المستخدم. أنت الآن في وضع الضيف.")

    def process_command(self, user_command):
        intent, entities = self.nlp.process(user_command)

        if intent == "change_mode":
            mode = entities.get("mode")
            if mode in self.modes:
                self.current_mode = mode
                self.modes[self.current_mode]()
            else:
                self.tts.speak("هذا الوضع غير متوفر.")

        elif intent == "control_device":
            device = entities.get("device")
            action = entities.get("action")
            if device and action:
                self.smart_home.control_device(device, action)
                self.tts.speak(f"تم {action} {device}.")
            else:
                self.tts.speak("لم أفهم أمر التحكم بالجهاز.")

        elif intent == "request_info":
            request_type = entities.get("type")
            if request_type == "time":
                self.tts.speak(f"الوقت الحالي هو {datetime.now().strftime('%I:%M %p')}")
            elif request_type == "weather":
                city = entities.get("city")
                if not city and self.current_user_id:
                    city = self.user_profiles[self.current_user_id].get("preferences", {}).get("city")
                weather_data = self.external_services.get_weather(city)
                self.tts.speak(weather_data)
            elif request_type == "news":
                news_data = self.external_services.get_news()
                self.tts.speak(news_data)

        elif intent == "update_preference":
            key = entities.get("key")
            value = entities.get("value")
            if key and value:
                self.update_user_preference(key, value)
            else:
                self.tts.speak("عذراً، لم أفهم التفضيل الذي تريد تحديثه.")

        elif intent == "shutdown":
            self.tts.speak("شكراً لاستخدامك Titapa. إلى اللقاء.")
            self.is_running = False
            self.gui.destroy()

        else:
            self.tts.speak("لم أفهم الأمر، حاول مرة أخرى.")

    def entertainment_mode(self):
        self.tts.speak("تم تفعيل وضع الترفيه. أوصي بفيلم 'رحلة إلى المريخ'.")

    def productivity_mode(self):
        self.tts.speak("تم تفعيل وضع الإنتاجية. مهامك اليومية جاهزة.")

    def smart_home_mode(self):
        self.tts.speak("تم تفعيل وضع المنزل الذكي. قل 'شغل الأنوار' أو 'أطفئ التكييف'.")

    def health_mode(self):
        self.tts.speak("تم تفعيل وضع الصحة واللياقة. لقد قطعت 5842 خطوة اليوم.")

class NLPController:
    def process(self, text):
        text = text.lower().strip()
        if any(word in text for word in ["ترفيه", "ترفيهي"]):
            return "change_mode", {"mode": "ترفيه"}
        if any(word in text for word in ["إنتاجية", "عمل"]):
            return "change_mode", {"mode": "إنتاجية"}
        if any(word in text for word in ["منزل", "منزل ذكي"]):
            return "change_mode", {"mode": "منزل ذكي"}
        if any(word in text for word in ["صحة", "لياقة"]):
            return "change_mode", {"mode": "صحة"}
        if any(word in text for word in ["لوحة", "تحكم", "رئيسية"]):
            return "change_mode", {"mode": "dashboard"}

        if any(word in text for word in ["شغل", "شغل لي", "افتح"]):
            action = "تشغيل"
            if "أنوار" in text or "لمبة" in text:
                return "control_device", {"action": action, "device": "lights"}
            if "تكييف" in text or "مكيف" in text:
                return "control_device", {"action": action, "device": "ac"}

        if any(word in text for word in ["أطفئ", "أغلق", "أغلق لي", "اطفي"]):
            action = "إيقاف"
            if "أنوار" in text or "لمبة" in text:
                return "control_device", {"action": action, "device": "lights"}
            if "تكييف" in text or "مكيف" in text:
                return "control_device", {"action": action, "device": "ac"}

        if any(word in text for word in ["كم الساعة", "الوقت"]):
            return "request_info", {"type": "time"}
        if any(word in text for word in ["ما الطقس", "طقس", "درجة الحرارة"]):
            city = None
            if "في" in text:
                city = text.split("في")[-1].strip()
            return "request_info", {"type": "weather", "city": city}
        if any(word in text for word in ["ما الأخبار", "أخبار"]):
            return "request_info", {"type": "news"}

        if "احفظ لي" in text:
            if "مدينتي" in text:
                parts = text.split("مدينتي")
                if len(parts) > 1:
                    city_name = parts[1].strip()
                    return "update_preference", {"key": "city", "value": city_name}

        if any(word in text for word in ["إيقاف", "اغلاق", "توقف"]):
            return "shutdown", None

        return "unknown", {}

class SmartHomeController:
    def __init__(self):
        self.devices = {"lights": False, "ac": False}

    def control_device(self, device, action):
        if device in self.devices:
            if action == "تشغيل":
                self.devices[device] = True
            elif action == "إيقاف":
                self.devices[device] = False

class ExternalServicesController:
    def get_weather(self, city=None):
        if not city:
            city = "المدينة"
        return f"الطقس في {city} الآن مشمس، ودرجة الحرارة 25 درجة مئوية."

    def get_news(self):
        return "العناوين الرئيسية: انخفاض أسعار النفط العالمية، وتطورات جديدة في قطاع التكنولوجيا."

    def get_status(self, city=None):
        weather_text = self.get_weather(city)
        if "مشمس" in weather_text:
            return "☀️ مشمس"
        elif "غائم" in weather_text:
            return "☁️ غائم"
        else:
            return "مقبول"

class TTSController:
    def __init__(self, titapa_core):
        self.titapa_core = titapa_core

    def speak(self, text):
        if self.titapa_core.gui:
            self.titapa_core.gui.display_message(f"Titapa: {text}", 'system')

if __name__ == "__main__":
    titapa = TitapaAI()
    gui = TitapaGUI(titapa)
    gui.mainloop()